In [100]:
import pandas as pd

In [101]:
# einlesen der Tripdaten als DataFrame
# Datei zu groß für Github upload, Downloadlink: https://nhts.ornl.gov/downloads -> 2017 Datasets -> Survey Data -> CSV 
# "trippub" Datei im Download des csv Ordners
df = pd.read_csv("C:\\Users\\thoma\\Desktop\\ev-modelling-repo\\Rohdaten\\NHTS_trips_raw.csv")

In [102]:
# erstellen leerer DataFrame
data = pd.DataFrame()

## Übernehmen relevanter Spalten und Umbennennung

In [103]:
# zusammenfügen von PERSONID und HOUSEID um einzigartige personenbezogene ID zu erlangen
data["ID"] = df["HOUSEID"].apply(lambda x: str(x)) + df["PERSONID"].apply(lambda x: str(x))
data["ID"] = data["ID"].apply(lambda x: int(x))
data["Type_day"] = df["TRAVDAY"]
data["TRPTRANS"] = df["TRPTRANS"]
data["Trip_no"] = df["TDTRPNUM"]
data["Whyfrom"] = df["WHYFROM"]
data["Whyto"] = df["WHYTO"]
data["Distance"] = df["VMT_MILE"]
data["Trip_duration"] = df["TRVLCMIN"]
data["Departure_hhmm"] = df["STRTTIME"]
data["Arrival_hhmm"]= df["ENDTIME"]

In [104]:
data.head()

,ID,Type_day,TRPTRANS,Trip_no,Whyfrom,Whyto,Distance,Trip_duration,Departure_hhmm,Arrival_hhmm
0,300000071,2,3,1,1,19,5.244,15,1000,1015
1,300000071,2,3,2,19,1,5.149,20,1510,1530
2,300000072,2,6,1,3,1,84.004,120,700,900
3,300000072,2,6,2,1,3,81.628,150,1800,2030
4,300000073,2,3,1,1,8,2.250,15,845,900


## Nur Wege betrachten die mit durch EV substituierbarem Fahrzeug zurückgelegt wurden
TRPTRANS = Trip Mode derived 


3 = Car


4 = SUV 


5 = Van 


6 = Pickup truck 

In [105]:
filt = (df["TRPTRANS"] > 2) & (df["TRPTRANS"] < 7)
data = data[filt]

# Zuordnung der Wochentage anpassen

Anstatt Sonntag = 1 bis Samstag = 7

Montag = 1 bis Sonntag = 7

In [106]:
data["Type_day"] = data["Type_day"].replace({1:7, 2:1, 3:2, 4:3, 5:4, 6:5, 7:6})

# Umrechnen der Wergzeiten von hhmm in Minuten

In [107]:
data.head(10)

,ID,Type_day,TRPTRANS,Trip_no,Whyfrom,Whyto,Distance,Trip_duration,Departure_hhmm,Arrival_hhmm
0,300000071,1,3,1,1,19,5.244,15,1000,1015
1,300000071,1,3,2,19,1,5.149,20,1510,1530
2,300000072,1,6,1,3,1,84.004,120,700,900
3,300000072,1,6,2,1,3,81.628,150,1800,2030
4,300000073,1,3,1,1,8,2.250,15,845,900
5,300000073,1,3,2,8,1,2.243,15,1430,1445
6,300000081,4,6,1,1,3,8.017,15,1115,1130
7,300000081,4,6,2,3,1,8.017,10,2330,2340
8,300000121,4,4,1,1,16,3.395,15,550,605
9,300000121,4,4,2,16,1,3.395,15,700,715


In [108]:
def to_minutes(hhmm):
    result = 0
    if len(str(hhmm)) == 4:
        hhmm = str(hhmm)
        hh = int(hhmm[0:2])
        mm = int(hhmm[2:4])
    if len(str(hhmm)) ==3:
        hmm = str(hhmm)
        hh = int(hmm[0])
        mm = int(hmm[1:3])
    if len(str(hhmm)) ==2:
        mm = str(hhmm)
        hh = 0
        mm = int(mm)
    if len(str(hhmm)) ==1:
        m = str(hhmm)
        hh = 0
        mm = int(m)
    result = hh*60+mm
    return result

In [109]:
data["Departure"] = data["Departure_hhmm"].apply(lambda x: to_minutes(x))

In [110]:
data["Arrival"] = data["Arrival_hhmm"].apply(lambda x: to_minutes(x))

In [111]:
data.head()

,ID,Type_day,TRPTRANS,Trip_no,Whyfrom,Whyto,Distance,Trip_duration,Departure_hhmm,Arrival_hhmm,Departure,Arrival
0,300000071,1,3,1,1,19,5.244,15,1000,1015,600,615
1,300000071,1,3,2,19,1,5.149,20,1510,1530,910,930
2,300000072,1,6,1,3,1,84.004,120,700,900,420,540
3,300000072,1,6,2,1,3,81.628,150,1800,2030,1080,1230
4,300000073,1,3,1,1,8,2.250,15,845,900,525,540


# Neuzuordnung Wegzwecke
-7 (I prefer not to answer) -> 99 (keine Angabe) 


-8 (I don't know) -> 99 (keine Angabe) 


-9 (Not ascertained) -> 99 (keine Angabe) 


01 (Regular home activities (chores, sleep)) -> 8 (nach Hause) 


02 (Work from home(paid)) -> 8 (nach Hause) 


03 (Work) -> 1 (Erreichen des Arbeitsplatzes) 


04 (Work-related meeting / trip) -> 2 (dienstlich/geschäftlich) 


05 (Volunteer acitivities (not paid)) -> 7 (Freizeitaktivität) 


06 (Drop off / pick up someone) -> 6 (Bringen/Holen/Begleiten von Personen) 


07 (Change type of transportation) -> 10 (anderer Zweck) 


08 (Attend school as a student) -> 3 (Erreichen der Ausbildungsstätte/Schule) 


09 (Attend child care) -> 10 (anderer Zweck) 


10 (Attend adult care) -> 10 (anderer Zweck) 


11 (Buy goods (groceries, clothes, appliances, gas)) -> 4 (Einkauf) 


12 (Buy services (dry cleaners, banking, service a car, pet care)) -> 5 (private Erledigungen) 


13 (Buy meals (go out for a meal, snack, carry-out)) -> 7 (Freizeitaktivität) 


14 (Other general errands (post office, library)) -> 5 (private Erledigungen)


15 (Recreational activities (visit parks, movies, bars, museums)) -> 7 (Freizeitaktivität) 


16 (Exercise (go for a jog, walk, walk the dog, go to the gym)) -> 7 (Freizeitaktivität) 


17 (Visit friends or relatives) -> 7 (Freizeitaktivität) 


18 Health care visit (medical, dental, therapy) -> 5 (private Erledigungen) 


19 Religious or other community activities	-> 7 (Freizeitaktivität) 


97 Something Else -> 10 (anderer Zweck)


In [112]:
reassignment = {-7:99, -8:99, -9:99, 1:8, 2:8, 3:1, 4:2, 5:7, 6:6, 7:10, 8:3, 9:10, 10:10, 11:4,
                12:5, 13:7, 14:5, 15:7, 16:7, 17:7, 18:5, 19:7, 97:10}

In [113]:
data["Whyto"] = data["Whyto"].replace(reassignment)
data["Whyfrom"] = data["Whyfrom"].replace(reassignment)

In [114]:
data["Whyto"].value_counts()

8     267833
7     166690
4     123504
1      87423
5      62990
6      53834
3      14104
2       9653
10      8950
99       182
Name: Whyto, dtype: int64

# Reduzieren der Zustände
auf: 
* Zuhause -> 1
* Arbeit -> 2 
* Einkaufen -> 3 
* Freizeit -> 4
* Sonstiges -> 5

### Zuordnung:
* Erreichen des Arbeitsplatzes 1 -> 2
* dienstlich/geschäftlich 2 -> 5  
* Erreichen der Ausbildungsstätte/Schule 3 -> 2 
* Einkauf (Dienstleistungen) 4 -> 3
* private Erledigung 5 -> 3
* Bringen/Holen/Begleiten von Personen 6 -> 5
* Freizeitaktivität 7 -> 4 
* nach Hause 8 -> 1
* Rückweg vom vorherigen Weg 9 -> zu letztem Zustand (in NHTS Datensatz nicht vorhanden)
* anderer Zweck 10 -> 5 Sonstiges
* keine Angabe 99 -> 5 Sonstiges (oder nicht berücksichtigen?)

In [115]:
reduzieren = {1:2, 2:5, 3:2, 4:3, 5:3, 6:5, 7:4, 8:1, 10:5, 99:5}

In [116]:
data["Whyto"] = data["Whyto"].replace(reduzieren)
data["Whyfrom"] = data["Whyfrom"].replace(reduzieren)

In [117]:
data["Whyto"].value_counts()

1    267833
3    186494
4    166690
2    101527
5     72619
Name: Whyto, dtype: int64

## (Modifikation) Nur Wegetagebücher betrachten, welche Zuhause starten

In [118]:
filt = (data["Trip_no"] == 1) & (data["Whyfrom"] != 1)
data_filt = data[filt]

In [119]:
data_filt

,ID,Type_day,TRPTRANS,Trip_no,Whyfrom,Whyto,Distance,Trip_duration,Departure_hhmm,Arrival_hhmm,Departure,Arrival
2,300000072,1,6,1,2,1,84.004,120,700,900,420,540
42,300000411,3,3,1,4,2,68.406,105,800,945,480,585
52,300000622,4,6,1,2,3,10.535,19,1535,1554,935,954
109,300001372,2,5,1,2,2,6.257,15,920,935,560,575
154,300001943,2,3,1,3,3,-1.000,395,405,1040,245,640
...,...,...,...,...,...,...,...,...,...,...,...,...
923073,407933311,6,3,1,3,3,1.954,10,1515,1525,915,925
923134,407934882,4,6,1,4,3,5.586,15,830,845,510,525
923142,407935201,5,4,1,3,5,1.990,5,530,535,330,335
923201,407936721,7,4,1,4,5,-1.000,43,1800,1843,1080,1123


In [120]:
IDs_to_drop = list(data_filt["ID"])

In [121]:
filt = data["ID"].isin(IDs_to_drop)
rows_to_drop = data[filt].index

In [122]:
data.drop(rows_to_drop, inplace=True)
data.reset_index(drop=True, inplace=True)

# Umrechnen Wegelänge

1mile = 1.60934km

In [123]:
data["Distance"] = data["Distance"].apply(lambda x: round(x*1.60934, 3) if x > 0 else -1)

In [124]:
data["Distance"]

0          8.439
1          8.286
2          3.621
3          3.610
4         12.902
           ...  
747212     1.257
747213     1.257
747214    15.389
747215    -1.000
747216    -1.000
Name: Distance, Length: 747217, dtype: float64

In [125]:
data.head()

,ID,Type_day,TRPTRANS,Trip_no,Whyfrom,Whyto,Distance,Trip_duration,Departure_hhmm,Arrival_hhmm,Departure,Arrival
0,300000071,1,3,1,1,4,8.439,15,1000,1015,600,615
1,300000071,1,3,2,4,1,8.286,20,1510,1530,910,930
2,300000073,1,3,1,1,2,3.621,15,845,900,525,540
3,300000073,1,3,2,2,1,3.610,15,1430,1445,870,885
4,300000081,4,6,1,1,2,12.902,15,1115,1130,675,690


## Zuordnen Startzeiten zu Zeitschritten
Schrittweite: t = 15 Minuten \
96 Zeitschritte pro Tag: W_SZ_t = [0, 96]

In [126]:
data["Departure_t"] = data["Departure"].apply(lambda x: round(x/15))

In [127]:
data.head(10)

,ID,Type_day,TRPTRANS,Trip_no,Whyfrom,Whyto,Distance,Trip_duration,Departure_hhmm,Arrival_hhmm,Departure,Arrival,Departure_t
0,300000071,1,3,1,1,4,8.439,15,1000,1015,600,615,40
1,300000071,1,3,2,4,1,8.286,20,1510,1530,910,930,61
2,300000073,1,3,1,1,2,3.621,15,845,900,525,540,35
3,300000073,1,3,2,2,1,3.610,15,1430,1445,870,885,58
4,300000081,4,6,1,1,2,12.902,15,1115,1130,675,690,45
5,300000081,4,6,2,2,1,12.902,10,2330,2340,1410,1420,94
6,300000121,4,4,1,1,4,5.464,15,550,605,350,365,23
7,300000121,4,4,2,4,1,5.464,15,700,715,420,435,28
8,300000121,4,4,3,1,4,1.704,10,730,740,450,460,30
9,300000121,4,4,4,4,1,1.632,10,805,815,485,495,32


## Ersetzen von fehlenden Distanzangaben

In [128]:
filt = data["Distance"] < 0
data[filt]

,ID,Type_day,TRPTRANS,Trip_no,Whyfrom,Whyto,Distance,Trip_duration,Departure_hhmm,Arrival_hhmm,Departure,Arrival,Departure_t
17,300000192,4,3,1,1,5,-1.0,15,1500,1515,900,915,60
22,300000292,2,4,1,1,3,-1.0,45,800,845,480,525,32
23,300000292,2,4,2,3,3,-1.0,10,1015,1025,615,625,41
24,300000292,2,4,3,3,1,-1.0,15,1050,1105,650,665,43
35,300000392,4,3,1,1,4,-1.0,5,1200,1205,720,725,48
...,...,...,...,...,...,...,...,...,...,...,...,...,...
747197,407942603,4,4,4,5,4,-1.0,22,1647,1709,1007,1029,67
747198,407942603,4,4,5,4,4,-1.0,23,1837,1900,1117,1140,74
747199,407942603,4,4,6,4,1,-1.0,10,2012,2022,1212,1222,81
747215,407943016,1,5,1,1,4,-1.0,26,1631,1657,991,1017,66


### Ermitteln durchschnittlicher Geschwindigkeit

In [129]:
filt = data["Distance"] > 0 
data_filt = data[filt]
av_speed = data_filt["Distance"].sum() / (data_filt["Trip_duration"].sum() / 60 )
av_speed

45.043108215941274

In [130]:
filt = (data["Distance"] > 0) & (data["Distance"] < 100) 
data_filt2 = data[filt]
av_speed2 = data_filt2["Distance"].sum() / (data_filt2["Trip_duration"].sum() / 60 )
av_speed2

39.26578840280872

### Tagtyp hat kaum Einfluss auf durchschnittliche Geschwindigkeit

In [131]:
filt = (data["Distance"] > 0) & (data["Distance"] < 100) & (data["Type_day"] < 6)
data_filt3 = data[filt]
av_speed_wd = data_filt3["Distance"].sum() / (data_filt3["Trip_duration"].sum() / 60 )
av_speed_wd

39.23599851283663

In [132]:
filt = (data["Distance"] > 0) & (data["Distance"] < 100) & (data["Type_day"] == 6)
data_filt_sa = data[filt]
av_speed_sa = data_filt_sa["Distance"].sum() / (data_filt_sa["Trip_duration"].sum() / 60 )
av_speed_sa

39.401443435250314

In [133]:
for i in range(len(data)):
    if data.at[i, "Distance"] < 0: 
        data.at[i, "Distance"] = (data.at[i, "Trip_duration"] / 60) * av_speed2

## Trips bei denen weder Distanz noch Wegedauer angegeben wurde

In [134]:
filt = data["Distance"] < 0
bad_data = data[filt]
bad_data

,ID,Type_day,TRPTRANS,Trip_no,Whyfrom,Whyto,Distance,Trip_duration,Departure_hhmm,Arrival_hhmm,Departure,Arrival,Departure_t
129,300002881,2,4,3,3,3,-5.889868,-9,1045,1055,645,655,43
187,300004022,3,4,2,4,4,-5.889868,-9,1800,1845,1080,1125,72
188,300004022,3,4,3,4,1,-5.889868,-9,1915,1916,1155,1156,77
4504,300068512,7,3,4,5,3,-5.889868,-9,1550,1610,950,970,63
12814,300183502,5,3,5,1,1,-5.889868,-9,1815,1825,1095,1105,73
...,...,...,...,...,...,...,...,...,...,...,...,...,...
738582,407728445,2,4,2,4,3,-5.889868,-9,1700,1745,1020,1065,68
739053,407740432,5,4,1,1,4,-5.889868,-9,1845,1900,1125,1140,75
739309,407748342,4,3,3,1,1,-5.889868,-9,1500,1505,900,905,60
742069,407818942,2,4,2,2,1,-5.889868,-9,1800,1810,1080,1090,72


In [135]:
ids_to_drop = list(bad_data["ID"])
filt = data["ID"].isin(ids_to_drop)
rows_to_drop = data[filt].index

In [136]:
rows_to_drop

Int64Index([   127,    128,    129,    130,    131,    132,    187,    188,
              4502,   4503,
            ...
            746521, 746522, 746523, 746524, 746525, 746526, 746527, 746528,
            746529, 746530],
           dtype='int64', length=846)

In [137]:
data.drop(rows_to_drop, inplace=True)
data.reset_index(drop=True, inplace=True)

## Speichern der Daten

In [138]:
data.to_csv(r'C:\Users\thoma\Desktop\ev-modelling-repo\Rohdaten\NHTS_trips_processed.csv', header=True, index=False) 